In [1]:
# Import Libraries
import pandas as pd
import time
from random import randint
from IPython.display import clear_output
import os
 
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
# Gather saved links
file_name = input("Enter file name: ") + '_offers.csv'

Enter file name:  remaining_links/network_engineer


In [3]:
links_df = pd.read_csv(file_name)

In [4]:
links_df.shape

(69, 1)

In [5]:
# Login credentials

with open('config.txt', 'r') as file:
    email, password = file.readlines()

In [6]:
# Login to LinkedIn

path = '/Users/kritixblaze/Developer/chromedriver_mac64/chromedriver'
driver = webdriver.Chrome(path)

/var/folders/4y/pcdt74h51_x2zz4w3hlpvzxh0000gn/T/ipykernel_59241/1557696076.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [7]:
driver.maximize_window() 
driver.minimize_window()
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [8]:
# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)

# User Credentials
driver.find_element('xpath', '//*[@id="username"]').send_keys(email[:-1])
driver.find_element('xpath', '//*[@id="password"]').send_keys(password)
time.sleep(1)

# Login button
driver.find_element('xpath', '//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)

In [9]:
# Function to collect info from links
def collect_info(link):
    driver.get(link)
    wait_time = randint(1, 4)
    clear_output(wait=True)
    print(f"Waiting {wait_time} seconds\n")
    time.sleep(wait_time)
    
    contents = driver.find_elements('class name', 'p5')
    
    if len(contents) > 0:
        
        if contents[0].text == 'Show more results':
            current_index = shuffled_df[shuffled_df['0'] == link].index[0] 
            print(f"{current_index}. Posting deleted")
            shuffled_df.drop(current_index, inplace=True)
            
        else:
            try:
                info = {}

                try:
                    info['Job Title'] = contents[0].find_element('tag name', 'h1').text

                except NoSuchElementException:
                    info['Job Title'] = "N/A"

                try:
                    info['Job Posting Link'] = link

                except NoSuchElementException:
                    info['Job Posting Link'] = "N/A"
            
                try:
                    company_url = contents[0].find_element('tag name', 'a').get_attribute('href')
            
                    if company_url[-6:] == '/life/':
                        company_url = company_url[:-5]
    
                    info['Company URL'] = company_url

                except NoSuchElementException:
                    info['Company URL'] = "N/A"
        
                try:
                    info['Company Name'] = contents[0].find_element('class name', 'jobs-unified-top-card__company-name').text

                except NoSuchElementException:
                    info['Company Name'] = "N/A"
            
                try:
                    info['Company Location'] = contents[0].find_element('class name', 'jobs-unified-top-card__bullet').text

                except NoSuchElementException:
                    info['Company Location'] = "N/A"

                try:
                    info['Workplace Type'] = contents[0].find_element('class name', 'jobs-unified-top-card__workplace-type').text

                except NoSuchElementException:
                    info['Workplace Type'] = "N/A"
            
                try:
                    spans = contents[0].find_elements('tag name', 'span') 
                    info['Company Size'] = list(filter(lambda info : 'employees' in info.text, spans))[0].text.split(" · ")[0]

                except NoSuchElementException:
                    info['Company Size'] = "N/A"
            
                job_infos.append(info)
                current_index = shuffled_df[shuffled_df['0'] == link].index[0] 
                print(f"{current_index}. {info}")
                shuffled_df.drop(current_index, inplace=True)

            except IndexError as e:
                print(e)
        
    else:
        print("No content found")

In [10]:
job_infos = []

In [11]:
shuffled_df = links_df.sample(frac=1, axis=1).sample(frac=1).reset_index(drop=True)

In [12]:
# Saving info to list
links_df['Output'] = shuffled_df['0'].apply(collect_info)

Waiting 4 seconds

list index out of range


In [13]:
len(job_infos)

47

In [14]:
shuffled_df.shape

(20, 1)

In [15]:
# Save any remaining links for future use
if shuffled_df.shape[0] > 0:
    shuffled_df.to_csv('remaining_links/' + file_name.split('/')[1], index=False)

In [16]:
output_df = pd.DataFrame(job_infos)

In [17]:
keyword = input("Enter keyword: ")

Enter keyword:  network engineer


In [18]:
output_df['Query'] = keyword

In [19]:
output_df.head()

,Job Title,Job Posting Link,Company URL,Company Name,Company Location,Workplace Type,Company Size,Query
0,Network Engineer - Cisco/Juniper,https://www.linkedin.com/jobs/view/3420233543/...,https://www.linkedin.com/company/webeltek/,Webeltek,Greater Kolkata Area,On-site,11-50 employees,network engineer
1,Network Engineer,https://www.linkedin.com/jobs/view/3404891994/...,https://www.linkedin.com/company/net-tech-grou...,Net Tech Group Inc,"Mumbai, Maharashtra, India",On-site,1-10 employees,network engineer
2,Network Engineer,https://www.linkedin.com/jobs/view/3442307739/...,https://www.linkedin.com/company/deutsche-bank/,Deutsche Bank,"Pune, Maharashtra, India",On-site,"10,001+ employees",network engineer
3,Network Engineer II,https://www.linkedin.com/jobs/view/3442885015/...,https://www.linkedin.com/company/stefanini/,Stefanini Group,"Hyderabad, Telangana, India",On-site,"10,001+ employees",network engineer
4,Systems Engineer - Oracle PL/SQL(1-2 years) - ...,https://www.linkedin.com/jobs/view/3418192007/...,https://www.linkedin.com/company/infosys/,Infosys,"Sahibzada Ajit Singh Nagar, Punjab, India",Hybrid,"10,001+ employees",network engineer


In [20]:
output_df.shape

(47, 8)

In [22]:
keyword.replace(' ', '_')

'network_engineer'

In [23]:
output_file = f"output_info/{keyword.replace(' ', '_')}.csv"

In [24]:
output_file

'output_info/network_engineer.csv'

In [25]:
if os.path.isfile(output_file):
    df = pd.read_csv(output_file)
    print(df.shape)
    output_df = pd.concat([df, output_df])

(237, 8)


In [26]:
output_df.shape

(284, 8)

In [27]:
output_df.to_csv(output_file, index=False)